In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/blog-authorship-corpus/blogtext.csv


In [3]:
df = pd.read_csv('/kaggle/input/blog-authorship-corpus/blogtext.csv')


In [4]:
df =  df.drop(['id', 'topic', 'sign', 'date'], axis=1)


In [5]:
df.to_csv('/kaggle/working//blogtext.csv', index=False)


In [6]:
import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from multiprocessing import Pool, cpu_count

# Download NLTK stop words data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet', '/usr/share/nltk_data')

from nltk.corpus import wordnet
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/



[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/w

In [7]:
def remove_special_char(text):
    pattern = r'[^a-zA-Z0-9\s]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text
def preprocessing(text):

    text = remove_special_char(text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def process_chunk(chunk):
    chunk['text'] = chunk['text'].apply(preprocessing)
    return chunk

def apply_preprocessing(File):
    chunks = pd.read_csv(File, chunksize=1000)

    pool = Pool(cpu_count())

    processed_chunks = pool.map(process_chunk, chunks)

    pool.close()
    pool.join()

    processed_df = pd.concat(processed_chunks)

    processed_df.to_csv(File, index=False)

In [8]:
apply_preprocessing('/kaggle/working//blogtext.csv')


In [9]:
df = df.dropna()


In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
import torch.nn as nn
import torch.optim as optim

In [12]:

# Load the pre-trained BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
# Freeze the BERT model layers
for param in bert_model.parameters():
    param.requires_grad = False

In [14]:
# Create the dataset class
class BlogDataset(Dataset):
    def __init__(self, df):
        self.texts = df['text'].tolist()
        self.genders = [0 if g == 'male' else 1 for g in df['gender'].tolist()]
        self.ages = df['age'].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        gender = self.genders[idx]
        age = self.ages[idx]

        input_ids = bert_tokenizer.encode(text, add_special_tokens=True, max_length=128, truncation=True)
        attention_mask = [1] * len(input_ids)

        return input_ids, attention_mask, gender, age

In [15]:
from torch.nn.utils.rnn import pad_sequence


In [16]:

def collate_fn(batch):
    input_ids, attention_mask, genders, ages = zip(*batch)
    input_ids = pad_sequence([torch.tensor(x) for x in input_ids], batch_first=True, padding_value=0)
    attention_mask = pad_sequence([torch.tensor(x) for x in attention_mask], batch_first=True, padding_value=0)
    genders = torch.tensor([x for x in genders]).float()
    ages = torch.tensor([x for x in ages]).float()
    return input_ids, attention_mask, genders, ages

In [46]:
# Create the model
class GenderAgeModel(nn.Module):
    def __init__(self):
        super(GenderAgeModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.gender_output = nn.Linear(768, 1)
        self.age_output_1 = nn.Linear(768, 128)
        self.age_output_2 = nn.Linear(128, 64)
        self.age_output_3 = nn.Linear(64, 1)  # Linear output layer
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids, attention_mask)[1]
        bert_output = self.dropout(bert_output)
        gender_output = self.sigmoid(self.gender_output(bert_output))
        age_output_1 = self.relu(self.age_output_1(bert_output))
        age_output_2 = self.relu(self.age_output_2(age_output_1))
        age_output_3 = self.age_output_3(age_output_2)  # No activation
        return gender_output, age_output_3

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
from accelerate import Accelerator

accelerator = Accelerator()

In [49]:
device = accelerator.device


In [50]:
df = pd.read_csv('/kaggle/working/blogtext.csv')
df =df.dropna()


In [51]:
# Create the DataLoader
batch_size = 1024

# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create the training dataset and data loader
train_dataset = BlogDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Create the test dataset and data loader
test_dataset = BlogDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


In [52]:
# Initialize the model and define the loss functions and optimizer
model = GenderAgeModel()
gender_criterion = nn.BCELoss()
age_criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


In [53]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [54]:
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

In [55]:
model, optimizer, train_loader, scheduler = accelerator.prepare(
    model, optimizer, train_loader, scheduler
)

In [ ]:
# Training loop
for epoch in range(9):
    for batch in train_loader:
        input_ids, attention_mask, gender_labels, age_labels = batch
        
        with accelerator.autocast():
            gender_output, age_output = model(input_ids, attention_mask)
        
        gender_labels = gender_labels.unsqueeze(1)
        age_labels = age_labels.unsqueeze(1)
        
        gender_loss = gender_criterion(gender_output, gender_labels)
        age_loss = age_criterion(age_output, age_labels)
        
        loss = gender_loss + age_loss
        accelerator.backward(loss)
        
        optimizer.step()
        optimizer.zero_grad()

    # Print the losses
    print(f"Epoch [{epoch+1}/{16}], Gender Loss: {gender_loss.item()}, Age Loss: {age_loss.item()}")

Epoch [1/16], Gender Loss: 0.6946000456809998, Age Loss: 589.0359497070312


In [ ]:
torch.save(model.state_dict(), '/kaggle/working/model.pth')
